In [1]:
from pyflink.table import TableEnvironment, EnvironmentSettings, DataTypes, StreamTableEnvironment, TableDescriptor, FormatDescriptor

In [2]:
from pyflink.table.schema import Schema

In [3]:
from pyflink.common.typeinfo import RowTypeInfo

In [4]:
from pyflink.datastream.connectors.kafka import KafkaSource, KafkaOffsetsInitializer, KafkaSink, DeliveryGuarantee, KafkaRecordSerializationSchema

In [5]:
from pyflink.common.serialization import SimpleStringSchema

In [6]:
from pyflink.datastream.formats.json import JsonRowDeserializationSchema, JsonRowSerializationSchema

In [7]:
from pyflink.datastream import StreamExecutionEnvironment

In [8]:
from pyflink.table.expressions import col as col_sp

In [9]:
from pyflink.common import Types

In [10]:
from pyflink.common.watermark_strategy import WatermarkStrategy

In [11]:
from pyflink.common.configuration import Configuration

In [12]:
from pyflink.common.restart_strategy import RestartStrategies

In [13]:
from pyflink.table import CsvTableSink

In [14]:
stream_env = StreamExecutionEnvironment.get_execution_environment()

In [15]:
stream_settings = EnvironmentSettings.new_instance()\
                                    .in_streaming_mode()\
                                    .build()

In [16]:
stream_tbl_env = StreamTableEnvironment.create(stream_execution_environment=stream_env,environment_settings=stream_settings)

In [17]:
stream_tbl_env.get_config().get_configuration().set_string("parallelism.default", "1")

In [18]:
stream_env.set_parallelism(1)

In [19]:
# Put Jars in: D:\Users\kritagya\anaconda3\envs\PyFlink\Lib\site-packages\pyflink

In [20]:
# stream_tbl_env.get_config().set('pipeline.jars','file:///C:/Users/krita/Udemy/PyFlink/flink-sql-connector-kafka-1.17.1.jar')

In [21]:
# stream_env.add_jars('pipeline.jars','file:///C:/Users/krita/Udemy/PyFlink/flink-sql-connector-kafka-1.17.1.jar')

In [22]:
kafka_tbl_sch_source = Schema.new_builder()\
                            .column('seller_id', data_type=DataTypes.STRING())\
                            .column('product',data_type=DataTypes.STRING())\
                            .column('quantity',data_type=DataTypes.INT())\
                            .column('product_price',data_type=DataTypes.DOUBLE())\
                            .column('sales_date',data_type=DataTypes.DATE())\
                            .build()

In [23]:
kafka_tbl_dis_source = TableDescriptor.for_connector(connector='kafka')\
                                    .schema(schema=kafka_tbl_sch_source)\
                                    .option('topic','productsales')\
                                    .option('properties.group.id','source-demo-1')\
                                    .option('properties.bootstrap.servers','localhost:9092')\
                                    .option('scan.startup.mode','earliest-offset')\
                                    .format('json')\
                                    .option('json.ignore-parse-errors','true')\
                                    .build()

In [24]:
stream_tbl_env.create_table('kafka_source_table', kafka_tbl_dis_source)

In [25]:
kafka_stream_tbl = stream_tbl_env.scan('kafka_source_table')

In [26]:
kafka_tbl_sch_sink = kafka_tbl_sch_source

In [29]:
kafka_tbl_dis_sink = TableDescriptor.for_connector('kafka')\
                                    .schema(schema=kafka_tbl_sch_sink)\
                                    .option('topic','productsales2')\
                                    .option('properties.group.id','source-demo-1')\
                                    .option('properties.bootstrap.servers','localhost:9092')\
                                    .option('scan.startup.mode','earliest-offset')\
                                    .format('json')\
                                    .option('json.ignore-parse-errors','true')\
                                    .build()

In [30]:
stream_tbl_env.create_table('kafka_sink_table',kafka_tbl_dis_sink)

In [32]:
kafka_stream_tbl.execute_insert('kafka_sink_table')